In [1]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, read_data
import polars as pl
from datetime import date

In [2]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

In [3]:
RPL.df

code,date,open,high,low,close,volume,accumulated_split_rate
i64,date,f64,f64,f64,f64,f64,f64
1301,2009-01-05,2100.0,2120.0,2060.0,2070.0,11500.0,10.0
1301,2009-01-06,2070.0,2090.0,2050.0,2070.0,27300.0,10.0
1301,2009-01-07,2050.0,2080.0,2050.0,2070.0,26900.0,10.0
1301,2009-01-08,2050.0,2070.0,2040.0,2060.0,19900.0,10.0
1301,2009-01-09,2070.0,2080.0,2050.0,2050.0,24300.0,10.0
…,…,…,…,…,…,…,…
9997,2024-12-02,758.0,760.0,750.0,752.0,125900.0,1.0
9997,2024-12-03,750.0,767.0,750.0,761.0,215900.0,1.0
9997,2024-12-04,759.0,764.0,755.0,760.0,162700.0,1.0


In [2]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

# PricelistPlをdailyからweeklyに変更する。
# 変更される列は、open, high, low, close, volumeのみ。
# 他のデータとjoinしやすいように、週足の月曜日と週足の金曜日の日付列をそれぞれ"date_mon"列と"date_fri"の列名で追加する
# with_columns_*メソッド等で後から追加された統計データ列は、すべて集約関数last()の実行結果である週最終営業日のデータに変換されるので要注意。
# PricelistPl.dfは日足データから週足データに変換される。
#def convert_daily_to_weekly(self) -> None:
RPL.convert_daily_to_weekly()
RPL.df

code,date,open,high,min,close,volume,accumulated_split_rate
i64,date,f64,f64,f64,f64,f64,f64
1301,2009-01-09,2100.0,2120.0,2040.0,2050.0,109900.0,10.0
1301,2009-01-16,2010.0,2050.0,1950.0,2000.0,68500.0,10.0
1301,2009-01-23,1980.0,2000.0,1910.0,1920.0,58900.0,10.0
1301,2009-01-30,1920.0,2040.0,1900.0,1990.0,81200.0,10.0
1301,2009-02-06,1970.0,2010.0,1910.0,1920.0,51400.0,10.0
…,…,…,…,…,…,…,…
9997,2024-11-22,721.0,736.0,720.0,734.0,377200.0,1.0
9997,2024-11-29,740.0,766.0,725.0,758.0,704000.0,1.0
9997,2024-12-06,758.0,770.0,750.0,766.0,731600.0,1.0


In [ ]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

from lib_dataprocess import KessanPl

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

items_df = KPL.get_settlement_performance_items_df("四")
# items_dfにpl.DataFrame.columns = ["code", "start_date", "end_date"]のpl.DataFrameを与えると、
# 各レコードのstart_dateからend_dateまでの株価騰落率の列を追加して返す
# *_pointは、起点(start)と終点(end)において、日足ローソクのどの時点の株価を起点、または終点とするか選択する。
#def get_stockprice_change_rate(self, 
#    items_df: pl.DataFrame,
#    start_point: Literal["open", "high", "low", "close"] = "open",
#    end_point: Literal["open", "high", "low", "close"] = "open"
#) -> pl.DataFrame:
df = RPL.get_stockprice_change_rate(items_df)
df

In [4]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

code = 1301

# codeで指定した証券コードの銘柄の指定した日付における最新の終値の株価を、(取引日, 株価)のタプルで返す
# def get_latest_dealingdate_and_price(self, code: int, valuation_date: date = date.today()) -> tuple:
tup = RPL.get_latest_dealingdate_and_price(code)
tup

(datetime.date(2024, 11, 29), 3910.0)

In [3]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

# 当日の始値~終値の騰落率列を追加
#def with_columns_daily_updown_rate(self) -> None:
RPL.with_columns_daily_updown_rate()
RPL.df

code,date,open,high,low,close,volume,accumulated_split_rate,daily_updown_rate
i64,date,f64,f64,f64,f64,f64,f64,f64
1301,2009-01-05,2100.0,2120.0,2060.0,2070.0,11500.0,10.0,-1.43
1301,2009-01-06,2070.0,2090.0,2050.0,2070.0,27300.0,10.0,0.0
1301,2009-01-07,2050.0,2080.0,2050.0,2070.0,26900.0,10.0,0.98
1301,2009-01-08,2050.0,2070.0,2040.0,2060.0,19900.0,10.0,0.49
1301,2009-01-09,2070.0,2080.0,2050.0,2050.0,24300.0,10.0,-0.97
…,…,…,…,…,…,…,…,…
9997,2024-12-16,785.0,789.0,781.0,786.0,103100.0,1.0,0.13
9997,2024-12-17,787.0,789.0,782.0,786.0,99400.0,1.0,-0.13
9997,2024-12-18,786.0,787.0,774.0,776.0,119100.0,1.0,-1.27


In [5]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

# colで指定した列のterm日の移動平均列を、25日移動平均であれば、ma25の
# ような列名(maの後ろに移動平均の日数)で追加する。
# termで指定した日数での移動平均が計算できない初期のレコードは、dropされてなくなる
# 全データで実施すると、かなりメモリを消費するので、200日移動平均などを取得する場合は、
# PricelistPl(filename).dfをfilterしてから実施しないとメモリが足りなくなるかもしれない。
# メモリが不足して実行プロセスがダウンした場合は、例外も出力されない。
# def with_columns_moving_average(self, term, col="close"):
RPL.with_columns_moving_average(25)
RPL.df

code,date,open,high,low,close,volume,accumulated_split_rate,ma25
i64,date,f64,f64,f64,f64,f64,f64,f64
1301,2009-02-09,1910.0,1930.0,1900.0,1900.0,10900.0,10.0,1984.8
1301,2009-02-10,1910.0,1950.0,1910.0,1930.0,8900.0,10.0,1979.2
1301,2009-02-12,1900.0,1920.0,1890.0,1900.0,12800.0,10.0,1972.4
1301,2009-02-13,1900.0,1930.0,1900.0,1930.0,8800.0,10.0,1966.8
1301,2009-02-16,1920.0,1940.0,1910.0,1940.0,9600.0,10.0,1962.0
…,…,…,…,…,…,…,…,…
9997,2024-11-25,740.0,740.0,734.0,737.0,169400.0,1.0,722.84
9997,2024-11-26,738.0,738.0,732.0,736.0,67800.0,1.0,723.48
9997,2024-11-27,736.0,736.0,725.0,732.0,70600.0,1.0,723.84


In [3]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

# 週でグループ化できるように、日付から週グループのインデックス列を追加する
# PricelistPl.dfに列を追加する
#def with_columns_weekid(self) -> None:
RPL.with_columns_weekid()
RPL.df

code,date,open,high,low,close,volume,accumulated_split_rate,week_id
i64,date,f64,f64,f64,f64,f64,f64,i16
1301,2009-01-05,2100.0,2120.0,2060.0,2070.0,11500.0,10.0,0
1301,2009-01-06,2070.0,2090.0,2050.0,2070.0,27300.0,10.0,0
1301,2009-01-07,2050.0,2080.0,2050.0,2070.0,26900.0,10.0,0
1301,2009-01-08,2050.0,2070.0,2040.0,2060.0,19900.0,10.0,0
1301,2009-01-09,2070.0,2080.0,2050.0,2050.0,24300.0,10.0,0
…,…,…,…,…,…,…,…,…
9997,2024-12-13,780.0,792.0,780.0,784.0,142700.0,1.0,831
9997,2024-12-16,785.0,789.0,781.0,786.0,103100.0,1.0,832
9997,2024-12-17,787.0,789.0,782.0,786.0,99400.0,1.0,832


In [3]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

# colで指定した列のwindow_sizeの移動zsocre列を、25日移動平均であれば、zs25の
# ような列名(zsの後ろに移動zscoreの日数)で追加する。
# PricelistPl.dfに直接列を追加。
#def with_columns_moving_zscore(self, window_size: int=25, col: str="volume") -> None:

RPL.with_columns_moving_zscore(window_size=5)
RPL.df.filter(pl.col("code")==9997)

code,date,open,high,low,close,volume,accumulated_split_rate,zs5
i64,date,f64,f64,f64,f64,f64,f64,f64
9997,2009-01-05,124.0,124.0,120.0,120.0,83100.0,0.5,null
9997,2009-01-06,122.5,124.5,120.5,122.0,602800.0,0.5,null
9997,2009-01-07,123.5,124.5,120.0,123.0,476400.0,0.5,null
9997,2009-01-08,122.5,122.5,118.0,120.5,229400.0,0.5,null
9997,2009-01-09,118.0,120.5,117.5,120.5,201800.0,0.5,-0.55
…,…,…,…,…,…,…,…,…
9997,2024-12-16,785.0,789.0,781.0,786.0,103100.0,1.0,-0.8
9997,2024-12-17,787.0,789.0,782.0,786.0,99400.0,1.0,-0.65
9997,2024-12-18,786.0,787.0,774.0,776.0,119100.0,1.0,-0.4
